## Pydantic Fundamentals

### Start without Pydantic

In [3]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name = "Robin", gender = "Male", age = 29)
person1

In [4]:
person1.name, person1.gender, person1.age

('Robin', 'Male', 29)

## Validation of our Person class

In [ ]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance (name, str):
            raise TypeError(f"Name must be a string, it cannot be {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

try:
    Person(name = 123, gender = "Male", age = 29)
except TypeError as err:
    print(err)

Name must be a string, it cannot be <class 'int'>


In [15]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance (name, str):
            raise TypeError(f"Name must be a string, it cannot be {type(name)}")
        self.name = name

        if not isinstance (gender, str):
            raise TypeError(f"Gender must be a string, it cannot be {type(gender)}")
        self.gender = gender

        if not isinstance (age, int):
            raise TypeError(f"Age must be an integer, it cannot be {type(age)}")
        if not 0 <= age < 120:
            raise TypeError(f"Age must be between 0 and 120, not {age}")
        self.age = age

    def __repr__(self):
        return f"Person(name={self.name}, gender={self.gender}, age={self.age})"

try:
    person2 = Person(name = "Joe", gender = "Male", age = -29)
except TypeError as err:
    print(err)

Age must be between 0 and 120, not -29


In [17]:
person3 = Person(name = "Jane", gender = "Female", age = 25)
person3

Person(name=Jane, gender=Female, age=25)

In [19]:
person3.age = -5
person3

Person(name=Jane, gender=Female, age=-5)

In [24]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance (name, str):
            raise TypeError(f"Name must be a string, it cannot be {type(name)}")
        self.name = name

        if not isinstance (gender, str):
            raise TypeError(f"Gender must be a string, it cannot be {type(gender)}")
        self.gender = gender

        self.age = age

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):

        if not isinstance (age, int):
            raise TypeError(f"Age must be an integer, it cannot be {type(age)}")
        if not 0 <= age < 120:
            raise TypeError(f"Age must be between 0 and 120, not {age}")
        self._age = age

    def __repr__(self):
        return f"Person(name={self.name}, gender={self.gender}, age={self.age})"


person4 = Person(name = "Natalie", gender = "Female", age = 50)
person4


Person(name=Natalie, gender=Female, age=50)

In [26]:
try:
    person4.age = -10
except Exception as e:
    print(e)

Age must be between 0 and 120, not -10


In [ ]:
try:
    Person(name = 34, gender = "Male", age = -29)
except TypeError as err:
    print(err)

Age must be between 0 and 120, not -29


In [29]:
try:
    Person(name = "Joey", gender = "Male", age = -29)
except TypeError as err:
    print(err)

Age must be between 0 and 120, not -29


## Validation using Pydantic

In [31]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person5 = Person(name = "Alice", gender = "Female", age = 30)
person5

Person(name='Alice', gender='Female', age=30)

In [33]:
person5.age = 35
person5

Person(name='Alice', gender='Female', age=35)

In [ ]:
### Issue

person5.age = "Thirty Five"
person5

Person(name='Alice', gender='Female', age='Thirty Five')

In [35]:
from pydantic import ValidationError

try:
    Person(name = 12345, gender = "Male", age = 30)
except ValidationError as e:
    print(e)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=12345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [37]:
### str "30" is converted to int 30

Person(name = "Eli", gender = "Male", age = "30")

Person(name='Eli', gender='Male', age=30)

In [39]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(
        validate_assignment = True)

person6 = Person(name = "Ompalompa", gender = "Male", age = 30)

try:
    person6.age = "Thrity"
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Thrity', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [41]:
try:
    Person(name = 777, gender = 67, age = "Thirty")
except ValidationError as e:
    print(e)

3 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=777, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=67, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [43]:
try:
    Person(name = "Sofia", gender = "Female", age = -45)
except ValidationError as e:
    print(e)

### Add age validation

In [45]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["Male", "Female", "Other"]
    age: int = Field(gt = -1, lt = 120)

    model_config = ConfigDict(
        validate_assignment = True)

In [46]:
try:
    Person(name = "Sofia", gender = "Polar Bear", age = -45)
except ValidationError as e:
    print(e)

2 validation errors for Person
gender
  Input should be 'Male', 'Female' or 'Other' [type=literal_error, input_value='Polar Bear', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-45, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization & De-serialization

In [49]:
# serialization

import json
with open("person6.json", "w") as file:
    json.dump(person6.model_dump(), file)

In [51]:
with open("person6.json", "r") as file:
    json_data = file.read()

print(json_data)

# de-serialization
Person.model_validate_json(json_data)


{"name": "Ompalompa", "gender": "Male", "age": 30}


Person(name='Ompalompa', gender='Male', age=30)